<span type="title">Play with Django</span> | <span type="update">2018-05-30</span> | <span type="version">1.4</span>

<span type="intro"><p type="card-text">这些是Django的基础知识，它并算不上一个教程或者指南，但是，麻雀虽小，五脏俱全。我在写完这篇笔记后就写出了一个用作链接跳转的Web服务，配合CGI跑在服务器上，已经工作几个月了。本文从配置环境、设置项目、设计模型、ADMIN可视化管理、模板语法，文件和使用、URL跳转管理、视图这几个方面介绍了Django。本文基于2.0.2版本的Django。</p></span>

# INIT

## 配置环境

```python
conda env list
conda create -n cmsite python=3.6
conda install -n cmsite django
activate cmsite
import django
django.__version__
deactivate cmsite

```

## 创建项目

使用以上语句配置环境，使用下面语句创建项目。

`django-admin startproject mysite`

```python

mysite/ #无意义，可删除
    manage.py #命令行工具
    mysite/ #Python 包
        __init__.py 
        settings.py #设置选项
        urls.py # URL映射
        wsgi.py # 服务器交互
```

It’s not a good idea to put any of this Python code within your Web server’s document root, because it risks the possibility that people may be able to view your code over the Web

```bash
python manage.py runserver 2333
```
运行测试服务器在本机的2333端口。

## 创建程序&`HELLO WORLD`

`python manage.py startapp polls`

程序和项目的区别在于，一个项目有多个程序，每个程序有自己的功能，一个程序可以属于多个项目。

```python
polls/
    __init__.py
    admin.py
    apps.py
    migrations/
        __init__.py
    models.py
    tests.py
    views.py #视图：返回HTML文档


#在views中创建文档映射

from django.http import HttpResponse
def index(request):
    return HttpResponse("Hello World!")

#创建urls.py文件，填入以下语句以创建一个urlpatterns，这个集合属于本程序

from django.urls import path

from . import views
urlpatterns = [
    path('',views.index,name='index'), #创建URL和执行映射函数的关系
    #path后的两个参数：route以及view，
    #view从自己的文件中找到对应函数并且返回文档
    #path后可以加name参数，方便查找和在模板中使用
]

#然而项目还不知道这个程序的urlpatterns，
#使用include('appname.urls')包含进来，
#添加到查找路径

from django.urls import path,include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('polls/',include('polls.urls')), #包含程序URLPATTERN

]
```
In a request to https://www.example.com/myapp/?page=3, the URLconf will also look for myapp/.



# SETTINGS

## 连接数据库

MYSQL 安装 mysqlclient 包。在总的setting配置文件中设置DATABASES字段。

```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        'USER':'',
        'PASSWORD':'',
        'HOST':'2.muninn.cn',
        'PORT':'3306',
        'NAME':'dbname',
    }
}
```
顺便设置一下TIME_ZONE，可以根据服务器和数据库时区设置。

使用 `python manage.py migrate` 来整合信息，程序会检查INSTALLED APPS并且为它们创建合适的表到数据库，可以看到，服务器多了以下表格;

```

+----------------------------+
| auth_group                 |
| auth_group_permissions     |
| auth_permission            |
| auth_user                  |
| auth_user_groups           |
| auth_user_user_permissions |
| django_admin_log           |
| django_content_type        |
| django_migrations          |
| django_session             |
```

## 管理和修改已安装的程序

在Setting中：

```python
INSTALLED_APPS = [
    'django.contrib.admin', 管理站点
    'django.contrib.auth', 认证站点
    'django.contrib.contenttypes', 网站内容框架
    'django.contrib.sessions', 用户会话
    'django.contrib.messages', 用户消息框架
    'django.contrib.staticfiles', 静态文件管理框架
]
```

项目和APP之间的联系是通过 APP.apps.AppConfig 类进行的，这个类是 django.apps.AppConfig 的子类。

# MODEL

模型是数据的结构，其包含数个字段以及其作用关系。

**创造模型**

在polls的model文件中插入：

```python
from django.db import models

#Each model is represented by a class that 
#subclasses django.db.models.Model

class Question(models.Model):
    # field’s name, your database will use it as the column name.
    question_text = models.CharField(max_length=200) 
    #an instance of a Field class
    pub_date = models.DateTimeField('date published') 
    #an instance of a Field class
    # 可以指定第一个字段为名称，比如 date publisted

class Choice(models.Model):
    question = models.ForeignKey(Question,on_delete=models.CASCADE)
    # That tells Django each Choice is related to a single Question. 
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
```

注意，两个类对应两个表，db.models有两个重要的类，其一为Model类，所有自己创建的类都继承于此，其二为Frield类，这是字段类型。在类中创建变量就相当于在SQL中创建列，并设置列属性。

**和项目联系起来**

当有了程序的模型之后，就要将模型添加到总的项目的setting 的 INSTALLED_APP中，每个程序是独立的，因此可以添加到别的项目中去。添加通过这样一个接口：polls.apps.PollsConfig, apps文件的这个类继承自django.apps.AppConfig，在这个类中定义了name。调用这个类可以创建程序的联系。

**提交更改**

当INSTALLED_APP设置好之后，就要执行 `python manage.py makemigrations polls` 为此程序创建一个临时文件，这个文件还不会执行。
```
Migrations for 'polls':
  polls/migrations/0001_initial.py:
    - Create model Choice
    - Create model Question
    - Add field question to choice
    
```
之后你可以通过 `python manage.py sqlmigrate polls 0001` 查看做出更改对应的SQL语句。

现在还没有真正更改，只有通过：

`python mange.py migrate` 才会连接数据库，并且将临时文件中的数据创建成表。

大概就是这三步，更改模型，添加到项目，makemigrations，migrate。


## MODEL API

```python
from django.db import models
from django.utils import timezone
import datetime

#Each model is represented by a class that subclasses django.db.models.Model

class Question(models.Model):

    # field’s name, your database will use it as the column name.
    question_text = models.CharField(max_length=200) #an instance of a Field class
    pub_date = models.DateTimeField('date published') #an instance of a Field class
    # 可以指定第一个字段为名称，比如 date publisted

    def __str__(self):
        return self.question_text

    def was_published_recently(self):
        return self.pub_date >= timezone.now() - datetime.timedelta(days=1)

class Choice(models.Model):
    
    question = models.ForeignKey(Question,on_delete=models.CASCADE)
    # That tells Django each Choice is related to a single Question. 
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)

    def __str__(self):
        return self.choice_text
```

这是上面那个例子，添加了str打印以及一些自定义的方法，注意，使用到了timezone这个工具，此工具会自动根据你在setting中设置的时区来返回时间以及一些时间函数。

使用 `python manage.py shell` 进入交互界面，这个界面包含很多 django 的变量，因此使用很方便；

对于模型而言，可以直接从app_name.models导入我们定义的类即可

from polls.models import Choice, Question

我们的类有以下方法：

```
>>> dir(Question)
['DoesNotExist', 'MultipleObjectsReturned', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_column_name_clashes', '_check_field_name_clashes', '_check_fields', '_check_id_field', '_check_index_together', '_check_local_fields', '_check_long_column_names', '_check_m2m_through_same_relationship', '_check_managers', '_check_model', '_check_model_name_db_lookup_clashes', '_check_ordering', '_check_swappable', '_check_unique_together', '_do_insert', '_do_update', '_get_FIELD_display', '_get_next_or_previous_by_FIELD', '_get_next_or_previous_in_order', '_get_pk_val', '_get_unique_checks', '_meta', '_perform_date_checks', '_perform_unique_checks', '_save_parents', '_save_table', '_set_pk_val', 'check', 'choice_set', 'clean', 'clean_fields', 'date_error_message', 'delete', 'from_db', 'full_clean', 'get_deferred_fields', 'get_next_by_pub_date', 'get_previous_by_pub_date', 'id', 'objects', 'pk', 'prepare_database_save', 'pub_date', 'question_text', 'refresh_from_db', 'save', 'save_base', 'serializable_value', 'unique_error_message', 'validate_unique', 'was_published_recently']
```

使用Qusetion.objects控制对象，它有以下方法：

```
>>> dir(Question.objects)
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_constructor_args', '_db', '_get_queryset_methods', '_hints', '_insert', '_queryset_class', '_set_creation_counter', '_update', 'aggregate', 'all', 'annotate', 'auto_created', 'bulk_create', 'check', 'complex_filter', 'contribute_to_class', 'count', 'create', 'creation_counter', 'dates', 'datetimes', 'db', 'db_manager', 'deconstruct', 'defer', 'difference', 'distinct', 'earliest', 'exclude', 'exists', 'extra', 'filter', 'first', 'from_queryset', 'get', 'get_or_create', 'get_queryset', 'in_bulk', 'intersection', 'iterator', 'last', 'latest', 'model', 'name', 'none', 'only', 'order_by', 'prefetch_related', 'raw', 'reverse', 'select_for_update', 'select_related', 'union', 'update', 'update_or_create', 'use_in_migrations', 'using', 'values', 'values_list']
```

**QuerySet、ChoiceSet、实例对象**

Question.objects.all() 返回所有行，它是一个 QuerySet对象

objects.filter(question_text__startswith="What") 过滤，根据首字母过滤

objects.get(pub_date__year = timzone.now().year) 获取元素

objects.get(id = 2)

get返回的是一个ChoiceSet对象，通过调用 o.get().choice_set 可以返回一个包含很多实例的列表，它有很多方法，比如 .all() .create() .count() .filter() .delete()

a = Question(question_text='',pub_date='') 创建一个实例

a.id 返回行对应的 id 列

a.question_text 返回对应的question_text列

a.question_text = ‘233’ 可以更改数据

a.save() 写回数据

a.was_published_recently() 私有方法


# ADMIN

`python manage.py createsuperuser`

localhost:port/admin to login


**model和ADMIN的连接**

在APP的admin文件中写下site.register(Q) 这表示可以用ADMIN 管理 Question 的数据。

```python
from django.contrib import admin
from .models import Question

admin.site.register(Question)
```

# 模板之URLS


模板属于VIEW层，有一套内嵌于HTML的语法。

先看看模板是如何和urls、views连接的。

在 polls 的 views.py 文件中，被 urlconf 调用的函数只返回两个结果，要么是HttpResponse，要么是404。

为了HTML和views解耦，在APP内新建一个templates的文件夹，在这个文件夹中建立同APP名文件夹，在子文件夹中建立index.html模板。这个模板是这样工作的：

首先在urls中被找到和当前请求所匹配的views函数，然后这个函数调用这个模板返回HTML内容。具体来说，对于希望捕获请求内容：

在 urls 中，用尖括号、类型、冒号、变量、函数和这条语句的别称来捕获内容，捕获的输入会通过view层的函数传递并进行处理。大概类似于这个样子：`detail(request=<HttpRequest object>, question_id=34)`

```python
polls/urls.py

from django.urls import path
from . import views

urlpatterns = [
    # ex: /polls/
    path('', views.index, name='index'),
    # ex: /polls/5/
    path('<int:question_id>/', views.detail, name='detail'),
    # ex: /polls/5/results/
    path('<int:question_id>/results/', views.results, name='results'),
    # ex: /polls/5/vote/
    path('<int:question_id>/vote/', views.vote, name='vote'),
]

# 而在 views 层中，存在和这些 urlconf 对应的函数以及其HTML返回
polls/views.py

def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)

def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)
```

顺便说一句，像这样写 `path('polls/latest.html', views.index)` 也可以，不过太傻了。

# 模板之VIEWS

在views.py中，需要进行模板的渲染并且作为HTML返回。以下是一个不太好的例子，在这个例子中，介绍了如何从数据库获取信息，并且返回信息的（虽然没什么HTML）。

```python
polls/views.py


from django.http import HttpResponse
from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    output = ', '.join([q.question_text for q in latest_question_list])
    return HttpResponse(output)

# Leave the rest of the views (detail, results, vote) unchanged
```

如果我们需要返回很多HTML，把HTML放在这里就太蠢了。我们需要在 urls-> views 之间搞一个东西，用来存放html模板。

在 polls文件夹下新建 templates 文件夹，在此文件夹下新建 polls 子文件夹，在此子文件夹下新建 index.html ， 因为在项目的 setting 中设置开启 appdir，所以程序会自动寻找 templates 文件夹。而放在子文件夹则避免多个程序搞混。

这个 index.html 的作用是什么呢，它提供一个可供更改变量的模板，用的时候只要替换变量就行了。它大概长这个样子：

```python
polls/templates/polls/index.html

    <ul>
    {% for question in latest_question_list %}
        <li>
            <a href="/polls/{{ question.id }}/">
                {{ question.question_text }}
            </a>
        </li>
    {% endfor %}
    </ul>
```
先不管里面的内容，大概意思就是留出变量的位置，现在关心在 views 中是如何进行渲染的。

一般而言，使用 django.template.loader 获取模板， temp = loader.get_template('address'),.当得到地址后，需要使用 temp.render(resquest,context) 进行渲染，把变量对照起来，并且直接返回HTML。如下：

```python
polls/views.py


from django.http import HttpResponse
from django.template import loader
from .models import Question

def index(request): #如果在url定义了变量的话，在这里应该作为参数传入
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    template = loader.get_template('polls/index.html')
    context = {
        'latest_question_list': latest_question_list,}
    return HttpResponse(template.render(context, request))
```

context 中的左边是在 index.html 中定义的替换变量，右边是值。

这样太傻了，有两个很方便的方法可以快速进行渲染。

`django.shortcuts.render(response,'polls/index.html',{"the_list":the_list})`

这个快捷方式用来进行 load 并且进行 render。

很多时候需要抛出404异常，如果查询不到数据的话，那么使用 raise django.http.Http404('reason') 是一个好主意，你可以使用下列方式捕获它：

```python
try:
    question = Question.objects.get(pk=question_id)
except Question.DoesNotExist:
    raise Http404("Question does not exist")
```

但是，这样的写法过于耦合了，因为在很多地方都依赖于各种Question的这个Error类型，因此使用一个函数来控制耦合很不错。

`django.shortcuts.get_object_or_404(Question， pk=question_id)`

这个函数等价于 
```python
from .model import Question
question = Question.get(pk=question_id)
render(resquest,'polls/details.html',{"quesion":question})
```

# 模板之TEMPLATES

最后看看模板本身，其定义的变量来自于 views 模块中 相应函数的 context 字典，而这个函数则从 urls 的 urlconf 对应语句中接受变量作为输入。

```python
polls/templates/polls/detail.html

<h1>{{ question.question_text }}</h1>
<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }}</li>
{% endfor %}
</ul>
```
这里面双括号括起来的就是变量，变量句点表示三种意思，首先django会去试图认为它是一个字典，如果不对，那么当作属性，如果还不对，那么当作索引，再如果，就跳过这段代码不执行。

单括号和百分比是语法的标志，常见的比如 for 和 if 与 python 语法很相似。需要注意，有一个叫做 {%url a b%} 的语法格式，这表示，从 urls 模块中接受名字为 a 的标签的变量，并且将值 b 传递给这个标签，最后返回那一坨标签。 这个很有意思，它的好处是，可以避免耦合，因为如果 urls 和 temps 耦合，而 temps 必定和 views 耦合（因为要传递字典），那么这就很不好。一个好的系统应该是 urls -> views -> temps -> views -> html

此外，再来看看这个 url 标签，默认状态，当我们要求一个 url，默认这个值是被传递出来的，而现在使用 url 语法，我们可以传递给这个变量，这多用于跳转URL的时候。

```python
bad example:(in .html)
<li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>

good example:(in .html)
    
<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>

polls.urls

# the 'name' value as called by the {% url %} template tag
path('specifics/<int:question_id>/', views.detail, name='detail'),
```

这样的话，不管你怎么更改 urls ，模板中都不用发生变化，而能仅仅根据 name 找到这条 url 了。

目前还有一个问题是，如果我有50个程序，那么这样的指代方式 `{% url 'name' variable%}` 不会混淆吗？ 当然会，因此要这样改动：

对于 `urls`， 添加一个叫做 app_name 的命名空间，比如：

`app_name = 'polls'` 这样的话，在 url 语句中进行如下修改 `{% url 'polls:detail' question.id %}` 就可以唯一确认那条URL了。

<br>
<hr>
<br>
**2018-05-30** SP0.6 环境设置、Hello World、模型、模型API、模板、URLs、视图 初步了解

<a href='https://docs.djangoproject.com/zh-hans/2.0/intro/tutorial03/'>官方指南</a>